In [1]:
import random 
import numpy as np

from get_data import load_data_small
from extract_features import extract_features
#from IPC_parser import *
from utils import construct_adjacency_matrix, encode_label_hierarchical, get_all_labels
from model import classify


   

class data_set:
	def __init__(self, X, Y, A):
		self.X = X
		self.Y = Y
		self.A = A


def construct_train_test_val_datasets(data, all_labels, A):
	N = len(data)
	indices = list(range(N))
	random.shuffle(indices)

	X = [data[k]['embedding'] for k in data.keys()]
	X = np.vstack(X)

	Y = [data[k]['labels'] for k in data.keys()]
	Y = encode_label_hierarchical(Y, all_labels)


	train = indices[:int(N*0.5)] # 70% data used for training
	test = indices[int(N*0.5) : int(N*0.8)] # 20% data used for testing
	val = indices[int(N*0.8):] # 10% data used for validation

	train = data_set(X[train, :], Y[train, :], A)
	test = data_set(X[test, :], Y[test, :], A)
	val = data_set(X[val, :], Y[val, :], A)

	return train, test, val






labeled_patent_data, unlabeled_patent_data = load_data_small(50000)
label_list = [labeled_patent_data[k]['labels'] for k in labeled_patent_data.keys()]
sections, classes, subclasses = get_all_labels(label_list) # returns A, B, .. | A01, A02, ..| A01B, A01C, ..
data = extract_features(labeled_patent_data, extractor = "tfidf+glove", K=300)

A = construct_adjacency_matrix(sections, classes, subclasses)

all_labels = sections + classes + subclasses
train, test, val = construct_train_test_val_datasets(data, all_labels, A)



Using TensorFlow backend.
100%|██████████| 7048/7048 [00:19<00:00, 369.12it/s]


In [2]:
results = classify(train, test, val, 30, [len(sections), len(classes), len(subclasses)]) # epoch num = 500


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3524 samples, validate on 1410 samples
Epoch 1/30
 - 2s - loss: 0.7333 - binary_crossentropy: 0.7331 - val_loss: 0.6549 - val_binary_crossentropy: 0.6547
Epoch 2/30
 - 2s - loss: 0.5733 - binary_crossentropy: 0.5731 - val_loss: 0.4717 - val_binary_crossentropy: 0.4715
Epoch 3/30
 - 1s - loss: 0.3898 - binary_crossentropy: 0.3896 - val_loss: 0.2975 - val_binary_crossentropy: 0.2973
Epoch 4/30
 - 1s - loss: 0.2356 - binary_crossentropy: 0.2354 - val_loss: 0.1720 - val_binary_crossentropy: 0.1718
Epoch 5/30
 - 1s - loss: 0.1359 - binary_crossentropy: 0.1357 - val_loss: 0.1046 - val_binary_crossentropy: 0.1044
Epoch 6/30
 - 2s - loss: 0.0957 - binary_crossentropy: 0.0955 - val_loss: 0.0839 - val_binary_crossentropy: 0.0837
Epoch 7/30
 - 2s - loss: 0.0793 - binary_crossentropy: 0.0792 - val_loss: 0.0701 - val

     average_precision_score 0.16058320931362743
     f1_score 0.1867424496194413
Test:
     average_precision_score 0.18649525713278972
     f1_score 0.1872844513853896


For case SECTION
Training dataset:
     average_precision_score 0.1601682287027833
     f1_score 0.014465866804171204

Testing dataset:
     average_precision_score 0.16089058077310645
     f1_score 0.019355993035442498


For case CLASS
Training dataset:
     average_precision_score 0.012430260990691154


/Users/xiahu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiahu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


     f1_score 0.000754963242719231

Testing dataset:
     average_precision_score 0.012402311918144742
     f1_score 0.0


For case SUBCLASS
Training dataset:
     average_precision_score 0.003366234725375165
     f1_score 0.0006365489072506033

Testing dataset:
     average_precision_score 0.003386832220250458
     f1_score 0.0
